<a href="https://colab.research.google.com/github/modyba10/PAF-MEJEC/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset : https://challenge2020.isic-archive.com

Importation des bibliothèques



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pydicom as dicom
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score

Analyse du Dataset

In [ ]:
df = pd.read_csv('train.csv')

#s0 = df.target[df.target.eq(0)].sample(60).index
#s1 = df.target[df.target.eq(1)].sample(60).index
#df = df.loc[s0.union(s1)]
#df['target'].value_counts()

indexes = df.index
age = []
for i in indexes:
    try:
        if df['target'][i] == 1:
            age.append(df['age_approx'][i])
    except:
        pass

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
sns.histplot(age)
plt.title('Distribution of age of people having melanoma')

plt.subplot(1, 2, 2)
sns.countplot(x=age)
plt.xlabel('Age')
plt.title('Count plot of age of people having malignant cancer')

plt.show()

df = df.rename(columns = {'anatom_site_general_challenge':'site'})

site = []
for i in indexes:
    try:
        if df['target'][i] == 1:
            site.append(df['site'][i])
    except:
        pass
sns.countplot(y = site)
sns.countplot(y = site, palette="rocket")
plt.title('Count of patients having cancer and the site it is located in')
plt.ylabel('Site')

plt.show()

Chargement des Images (DICOM)

In [ ]:
# Test loading DICOM Images

image = 'train/' + df['image_name'][1501] +'.dcm'
ds = dicom.dcmread(image)
plt.imshow(ds.pixel_array)

# load all the Images

images = []
for x in df['image_name']:
    image = 'train/' + x +'.dcm'
    ds = dicom.dcmread(image)
    pixels = ds.pixel_array
    images.append(pixels.flatten())

# normalise number of pixels for every image (padding)

images = tf.keras.preprocessing.sequence.pad_sequences(
  images,
  maxlen = 720,
  dtype = "int32",
  padding = "pre",
  truncating = "pre",
  value = 0
)

Test

In [ ]:
# Test dataset

test = df.tail(50)
test.head()

# padding test

test_images = []
count = 0
for x in test['image_name']:
    image = 'train/' + x + '.dcm'
    ds = dicom.dcmread(image)
    pixels = ds.pixel_array
    test_images.append(pixels.flatten())
    count += 1
    print(count)


test_images = tf.keras.preprocessing.sequence.pad_sequences(
    test_images,
    maxlen=720,
    dtype="int32",
    padding="pre",
    truncating="pre",
    value=0
)

Entrainement des différents algorithmes de ML

In [ ]:
# TRAINING

"""
X = images (list containing images in pixel format)
y = np.array(df[‘target’]) (values that state whether the lesion in the image is benign or malignant)
"""

# 1) logistic regression

from sklearn.linear_model import LogisticRegression
X = images
y = np.array(df['target'])
classifier_lr = LogisticRegression()
classifier_lr.fit(X,y)
X_test = test_images
y_test = np.array(test['target'])
y_pred_lr = classifier_lr.predict(X_test)
print('Accuracy Score: ',accuracy_score(y_test,y_pred_lr))
print('Confusion Matrix: \n',confusion_matrix(y_test,y_pred_lr))

# 2) SVM

from sklearn import svm
X = images
y = np.array(df['target'])
classifier_svm = svm.SVC()
classifier_svm.fit(X,y)
X_test = test_images
y_test = np.array(test['target'])
y_pred_svm = classifier_svm.predict(X_test)
print('Accuracy Score: ',accuracy_score(y_test,y_pred_svm))
print('Confusion Matrix: \n',confusion_matrix(y_test,y_pred_svm))

# 3) Decision Tree

from sklearn.tree import DecisionTreeClassifier
X = images
y = np.array(df['target'])
classifier_dt = DecisionTreeClassifier()
classifier_dt.fit(X,y)
X_test = test_images
y_test = np.array(test['target'])
y_pred_dt = classifier_dt.predict(X_test)
print('Accuracy Score: ',accuracy_score(y_test,y_pred_dt))
print('Confusion Matrix: \n',confusion_matrix(y_test,y_pred_dt))

# 4) Random Forest

from sklearn.ensemble import RandomForestClassifier
X = images
y = np.array(df['target'])
classifier_rf = RandomForestClassifier()
classifier_rf.fit(X,y)
X_test = test_images
y_test = np.array(test['target'])
y_pred_rf = classifier_rf.predict(X_test)
print('Accuracy Score: ',accuracy_score(y_test,y_pred_rf))
print('Confusion Matrix: \n',confusion_matrix(y_test,y_pred_rf))

# 5) Adaptative Boosting

from sklearn.ensemble import AdaBoostClassifier
X = images
y = np.array(df['target'])
classifier_ab = AdaBoostClassifier()
classifier_ab.fit(X,y)
X_test = test_images
y_test = np.array(test['target'])
y_pred_ab = classifier_ab.predict(X_test)
print('Accuracy Score: ',accuracy_score(y_test,y_pred_ab))
print('Confusion Matrix: \n',confusion_matrix(y_test,y_pred_ab))